Goal: Mosaic the landsat scenes for five variables:

    - spring onset
    - automn onset
    - peak NDVI
    - Red
    - NIR 
source of the data [ABoVE: Annual Phenology Derived from Landsat across the ABoVE Core Domain, 1984-2014](https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=1698) 

Note it takes so much memory! If you run into memory issue use [this package](https://github.com/bluegeo/dask-mosaic)

In [1]:
import xarray as xr
import pandas as pd
import rasterio
from rasterio.merge import merge
import glob
import numpy as np


In [2]:
# Read raw data

dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/"

in_dir = (
    "/data/ABOVE/ABoVE_Final_Data/landsat/phenology/orders/"
    "dc6b1f56b7619c37e1a4b6fec7ce3dcc/Annual_Seasonality_Greenness/data/"
)
out_dir = dir + "data/processed_data/landsat/mosaic/"


Refer to [HERE](https://daac.ornl.gov/ABOVE/guides/Annual_Seasonality_Greenness.html) for data structur. 

Each variable (spring onset and autumn onset) are recorded as a file with 31 bands where each band is a year from 1984 to 2014

In [4]:
years = pd.date_range(start="1984", end="2015", freq="A").year

# Spring onset and autumn onset
var_names = ["spr", "aut"]

for i in var_names:
    for j in np.arange(0, len(years)):

        # Read all the file names
        fnames = glob.glob(in_dir + "*pheno." + i + ".tif")

        src_files_to_mosaic = []

        # Open all files
        for fp in fnames:
            src = rasterio.open(fp)
            src_files_to_mosaic.append(src)

        # Start mosaicing
        print(
            "Mosaicing sesonal,year:"
            + str(years[j])
            + ",var:"
            + i
            + ",#scenes:"
            + str(len(fnames))
        )

        mosaic, out_trans = merge(
            src_files_to_mosaic, indexes=[j + 1]
        )  # note bands start from 1
        out_meta = src.meta.copy()
        # Update the metadata
        out_meta.update(
            {
                "driver": "GTiff",
                "height": mosaic.shape[1],
                "width": mosaic.shape[2],
                "transform": out_trans,
                "crs": src.crs,
                "count": 1,
            }
        )

        # Save the mosaic
        with rasterio.open(
            out_dir + "mosaic_" + i + "_" + str(years[j]) + ".tif",
            "w",
            **out_meta,
            compress="lzw"
        ) as dest:
            dest.write(mosaic)

        # Free memory
        del mosaic, out_meta, src_files_to_mosaic


Mosaicing sesonal,year:1985,var:spr,#scenes:169
Mosaicing sesonal,year:1986,var:spr,#scenes:169
Mosaicing sesonal,year:1987,var:spr,#scenes:169
Mosaicing sesonal,year:1988,var:spr,#scenes:169
Mosaicing sesonal,year:1989,var:spr,#scenes:169
Mosaicing sesonal,year:1990,var:spr,#scenes:169
Mosaicing sesonal,year:1991,var:spr,#scenes:169


Mosaicing TM5 data. These data are from 1984 to 2011. 

In [3]:
years_tm5 = pd.date_range(start="1984", end="2012", freq="A").year

b_lst = [1, 3, 4]  # Bands to be selected for mosaicing (i.e. ndvi,red, nir)
b_lst_names = ["NDVI", "Red", "NIR"]

for i in np.arange(0, len(years_tm5)):
    for j in np.arange(0, len(b_lst)):

        # Read all the file names
        fnames = glob.glob(in_dir + "*tm5." + str(years_tm5[i]) + ".tif")
        src_files_to_mosaic = []

        # Open all files
        for fp in fnames:
            src = rasterio.open(fp)
            src_files_to_mosaic.append(src)

        # Start mosaicing
        print(
            "Mosaicing et5,year:"
            + str(years_tm5[i])
            + ",var:"
            + str(b_lst_names[j])
            + ",#scenes:"
            + str(len(fnames))
        )

        mosaic, out_trans = merge(src_files_to_mosaic, indexes=[b_lst[j]])
        out_meta = src.meta.copy()
        # Update the metadata
        out_meta.update(
            {
                "driver": "GTiff",
                "height": mosaic.shape[1],
                "width": mosaic.shape[2],
                "transform": out_trans,
                "crs": src.crs,
                "count": 1,
            }
        )

        # Save the mosaic
        with rasterio.open(
            out_dir + "mosaic_" + b_lst_names[j] + "_" + str(years_tm5[i]) + ".tif",
            "w",
            **out_meta,
            compress="lzw"
        ) as dest:
            dest.write(mosaic)

        # Free memory
        del mosaic, out_meta, src_files_to_mosaic


Mosaicing et5,year:1984,var:NDVI,#scenes:9
Mosaicing et5,year:1984,var:Red,#scenes:9
Mosaicing et5,year:1984,var:NIR,#scenes:9
Mosaicing et5,year:1985,var:NDVI,#scenes:9
Mosaicing et5,year:1985,var:Red,#scenes:9
Mosaicing et5,year:1985,var:NIR,#scenes:9
Mosaicing et5,year:1986,var:NDVI,#scenes:9


KeyboardInterrupt: 

Mosaicing ETM data. These data ranges from 1999 to 2014

In [32]:
years_etm = pd.date_range(start="1999", end="2015", freq="A").year
b_lst = [1, 3, 4]
b_lst_names = ["NDVI", "Red", "NIR"]

for i in np.arange(0, len(years_etm)):
    for j in np.arange(0, len(b_lst)):

        # Read all the file names
        fnames = glob.glob(in_dir + "*etm." + str(years_etm[i]) + ".tif")
        src_files_to_mosaic = []

        # Open all files
        for fp in fnames:
            src = rasterio.open(fp)
            src_files_to_mosaic.append(src)
        # Start mosaicing
        print(
            "Mosaicing ETM,year:"
            + str(years_etm[i])
            + ",var:"
            + str(b_lst_names[j])
            + ",#scenes:"
            + str(len(fnames))
        )

        mosaic, out_trans = merge(src_files_to_mosaic, indexes=[b_lst[j]])
        out_meta = src.meta.copy()
        # Update the metadata
        out_meta.update(
            {
                "driver": "GTiff",
                "height": mosaic.shape[1],
                "width": mosaic.shape[2],
                "transform": out_trans,
                "crs": src.crs,
                "count": 1,
            }
        )
        # Save the mosaic
        with rasterio.open(
            out_dir + "mosaic_" + b_lst_names[j] + "_" + str(years_etm[i]) + ".tif",
            "w",
            **out_meta,
            compress="lzw"
        ) as dest:
            dest.write(mosaic)
        del mosaic, out_meta, src_files_to_mosaic


No file for in ETM data year:1999,var:NDVI
No file for in ETM data year:2000,var:NDVI
No file for in ETM data year:2001,var:NDVI
No file for in ETM data year:2002,var:NDVI
No file for in ETM data year:2003,var:NDVI
No file for in ETM data year:2004,var:NDVI
No file for in ETM data year:2005,var:NDVI
No file for in ETM data year:2006,var:NDVI
No file for in ETM data year:2007,var:NDVI
No file for in ETM data year:2008,var:NDVI
No file for in ETM data year:2009,var:NDVI
No file for in ETM data year:2010,var:NDVI
No file for in ETM data year:2011,var:NDVI
No file for in ETM data year:2012,var:NDVI
No file for in ETM data year:2013,var:NDVI
